# Compute model bias

In [ ]:
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
import simplejson
import tqdm
import xarray as xr

In [ ]:
def export(name, out):
    with open(f"../data/{name}.json", "w") as f:
        simplejson.dump(out, f, sort_keys=True, ignore_nan=True)

## Reference data

### E-OBS

In [ ]:
eobs_tas = xr.open_dataset("reference/tg_ens_mean_0.1deg_reg_v30.0e_REMAP_MEAN_1971-2000.nc")
eobs_tas = (eobs_tas["tg"] + 273.15).squeeze(drop=True)  # convert to kelvin

eobs_tas_var = xr.open_dataset("reference/tg_ens_mean_0.1deg_reg_v30.0e_REMAP_VAR_1971-2000.nc")
eobs_tas_var = eobs_tas_var["tg"].squeeze(drop=True)

eobs_pr = xr.open_dataset("reference/rr_ens_mean_0.1deg_reg_v30.0e_REMAP_MEAN_YEARSUM_1971-2000.nc")
eobs_pr = eobs_pr["rr"].squeeze(drop=True)

eobs_pr_var = xr.open_dataset("reference/rr_ens_mean_0.1deg_reg_v30.0e_REMAP_VAR_YEARSUM_1971-2000.nc")
eobs_pr_var = eobs_pr_var["rr"].squeeze(drop=True)

eobs = xr.merge([
    eobs_tas.rename("tas"),
    eobs_pr.rename("pr"),
    eobs_tas_var.rename("tas_var"),
    eobs_pr_var.rename("pr_var")
]).rio.write_crs(4326)

### ERA5

In [ ]:
era5_tas = xr.open_dataset("reference/era5_1971_2000_tas_MEAN_REMAP.nc")
era5_tas = era5_tas["t2m"].squeeze(drop=True)

era5_tas_var = xr.open_dataset("reference/era5_1971_2000_tas_VAR_REMAP.nc")
era5_tas_var = era5_tas_var["t2m"].squeeze(drop=True)

era5_pr = xr.open_dataset("reference/era5_1971_2000_total_precip_YEARSUM_REMAP.nc")
era5_pr = era5_pr["tp"].squeeze(drop=True)

era5_pr_var = xr.open_dataset("reference/era5_1971_2000_total_precip_VAR_YEARSUM_REMAP.nc")
era5_pr_var = era5_pr_var["tp"].squeeze(drop=True)

era5 = xr.merge([
    era5_tas.rename("tas"),
    era5_pr.rename("pr"),
    era5_tas_var.rename("tas_var"),
    era5_pr_var.rename("pr_var")
]).rio.write_crs(4326)

## Model data

In [ ]:
def preprocess(ds):
    name, _ = os.path.splitext(os.path.basename(ds.encoding["source"]))
    _, _, gcm, _, ens, rcm, *_ = name.split("_")
    ds = ds.drop_vars(['time', 'time_bnds'])
    ds =  ds.expand_dims({"model": [f"{gcm} {rcm} {ens}"]})
    if "height" in ds.coords:
        ds = ds.drop_vars(['height'])
    return ds

models = xr.open_mfdataset("models-hist/*1971_2000*.nc4", preprocess=preprocess).squeeze().load()
models["pr"] = models["pr"] * 86400.
models = models.rio.write_crs(4326)

In [ ]:
export("metadata", {
    "models": [dict(zip(["gcm", "rcm", "ens"], m.split(" "))) for m in models["model"].values],
    "attrs": {
        "pr": {
            "name": "precipitation",
            "bias": {
                "unit": "%",
                "period": "1971-2000"
            }
        },
        "tas": {
            "name": "temperature",
            "bias": {
                "unit": "°C",
                "period": "1971-2000"
            }
        }
    }
})

## Zonal statistics of bias


Run the regions-euro-cordex notebook to generate the regions file.

In [ ]:
regions = gpd.read_file("../data/regions.geojson")

In [ ]:
def zonal_stats(bias, regions, func):
    return {
        nuts_id: func(bias.rio.clip([region.geometry], all_touched=True))
        for nuts_id, region in tqdm.tqdm(regions.set_index("id").iterrows())
    }

In [ ]:
def bias(ref):
    """Bias as (model - reference)"""
    return xr.Dataset({
        "tas": (models["tas"] - ref["tas"]),
        "pr": (models["pr"] - ref["pr"] + 0.00001) / (ref["pr"] + 0.00001) * 100.  # in percent
    })

def weighted_median(data):
    area_weights = np.cos(np.deg2rad(data["latitude"]))
    median = data.weighted(area_weights).quantile(0.5, dim=["latitude", "longitude"])
    return {
        "tas": median["tas"].values.round(3).tolist(),
        "pr": median["pr"].values.round(2).tolist()
    }

bias_era5 = zonal_stats(bias(era5), regions, weighted_median)
bias_eobs = zonal_stats(bias(eobs), regions, weighted_median)

In [ ]:
# Rank model by total bias (all variables), combine biases with method of
# Reichler and Kim (2008), BAMS.

# TODO This implementation is not consistent with the computation of bias above as it rather
# considers "error", i.e. bias independent of sign without the possibility of "compensation"
# during the spatial aggregation. It also need adapting to the use of the median in the
# spatial aggreation used for the bias values.

def normalized_error(ref):
    return xr.Dataset({
        # Equation (1), inner
        v: (models[v] - ref[v])**2 / ref[v + "_var"]
        for v in ["tas", "pr"]
    })

def rank(data):
    return {}  # TODO omit until we've figured out the consistency issues
    area_weights = np.cos(np.deg2rad(data["latitude"]))
    # Equation (1) outer (weighted sum)
    e2vm = data.weighted(area_weights).sum(dim=["latitude", "longitude"])
    # Equation (2)
    i2vm = e2vm / e2vm.mean(dim="model")
    # Equation (3)
    i2m = 0.5 * (i2vm["tas"].values + i2vm["pr"].values)
    # Save ranks rather than I2m values (more compact in output)
    return {"rank": i2m.argsort().argsort().tolist()}

rank_era5 = zonal_stats(normalized_error(era5), regions, rank)
rank_eobs = zonal_stats(normalized_error(eobs), regions, rank)

In [ ]:
export("bias-era5", {key: {**bias_era5[key], **rank_era5[key]} for key in bias_era5.keys()})
export("bias-eobs", {key: {**bias_eobs[key], **rank_eobs[key]} for key in bias_eobs.keys()})